In [1]:
import pandas as pd
import seaborn as sb

# df = pd.read_csv('/Users/sachacollinson/Downloads/fake_transactional_data_24.csv')
# df_NonNumeric_Mapping = pd.read_excel('/Users/sachacollinson/Downloads/Location_value_pairs.xlsx')
df = pd.read_csv('/Users/thanaparleelasathapornkun/Library/Mobile Documents/com~apple~CloudDocs/Term2/python_working/Mini_Project_python_working/fake_transactional_data_24.csv', encoding='utf-8')
df_NonNumeric_Mapping = pd.read_excel('/Users/thanaparleelasathapornkun/Library/Mobile Documents/com~apple~CloudDocs/Term2/python_working/Mini_Project_python_working/Location_value_pairs.xlsx')

df = df.rename(columns={"from_totally_fake_account": "AccountNumber", "monopoly_money_amount": "MoneyTransferAmount", "to_randomly_generated_account": "TransferredLocationAccount", "not_happened_yet_date": "TransactionDate"})
# from_totally_fake_account  = AccountNumber
# monopoly_money_amount = MoneyTransferAmount
# to_randomly_generated_account = TransferredLocationAccount
# not_happened_yet_date = TransactionDate
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format="%d/%m/%Y").dt.date
df = df.sort_values(by=['TransactionDate'])

#non-numeric and numeric value
df_Transferred_numeric = df[df['TransferredLocationAccount'].apply(lambda x: str(x).isnumeric())].copy()
df_Transferred_Non_Numeric = df[df['TransferredLocationAccount'].apply(lambda x: str(x).isnumeric()==False)].copy()

In [2]:

df_NonNumeric_Mapping['Key'] = df_NonNumeric_Mapping['Key'].str.strip()
mapping_dict = df_NonNumeric_Mapping.to_dict(orient='records')
mapped_dict = {entry['Key']: entry['Value'] for entry in mapping_dict}

# # Create a new column by mapping values from the 'key' column using the mapping dictionary
df_Transferred_Non_Numeric['CategoryTransferredLocationAccount'] = df_Transferred_Non_Numeric['TransferredLocationAccount'].map(mapped_dict)

# # Print the DataFrame with the new column
print(df_Transferred_Non_Numeric)
 
frames = [df_Transferred_Non_Numeric,df_Transferred_numeric]
df2 = pd.concat(frames)
df2['CategoryTransferredLocationAccount'].fillna('Numeric', inplace=True)
print(df2)
#test

          AccountNumber  MoneyTransferAmount TransferredLocationAccount  \
0               10371.0                 4.00                     CINEMA   
13385           98994.0                 2.65                COFFEE_SHOP   
13384           98994.0                 2.20        A_LOCAL_COFFEE_SHOP   
13383           34189.0                14.50                        BAR   
13382           86417.0                 4.35                COFFEE_SHOP   
...                 ...                  ...                        ...   
10137338        35802.0                 1.95        HIPSTER_COFFEE_SHOP   
10137339        35802.0                 2.55        GOURMET_COFFEE_SHOP   
10137341        48432.0                 2.45                COFFEE_SHOP   
10137342        35802.0                 5.30                COFFEE_SHOP   
10148279        53444.0                 2.65        A_LOCAL_COFFEE_SHOP   

         TransactionDate CategoryTransferredLocationAccount  
0             2025-01-01             

/var/folders/__/6m3ls1tn1m5btytc8x1n4_j40000gn/T/ipykernel_5527/1807506615.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['CategoryTransferredLocationAccount'].fillna('Numeric', inplace=True)


Working out the Recency, Frequency and Monetary Values and Scores


In [3]:
# Ensure TransactionDate is in datetime format
df2['TransactionDate'] = pd.to_datetime(df['TransactionDate'], dayfirst=True)  # Adjusting for day-first format
end_date = pd.Timestamp('2025-12-31')

# RFM Calculation
rfm = df2.groupby('AccountNumber').agg({
    'TransactionDate': lambda x: (end_date - x.max()).days,
    'AccountNumber': 'count',
    'MoneyTransferAmount': 'sum'
}).rename(columns={'TransactionDate': 'Recency',
                   'AccountNumber': 'Frequency',
                   'MoneyTransferAmount': 'MonetaryValue'})

# Scoring RFM values
# Quantiles can be adjusted as needed
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])
quantiles = quantiles.to_dict()

def rfm_score(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

# Apply the scoring
rfm['R_Score'] = rfm['Recency'].apply(rfm_score, args=('Recency', quantiles,))
rfm['F_Score'] = rfm['Frequency'].apply(lambda x: 4 if x <= quantiles['Frequency'][0.25] else 3 if x <= quantiles['Frequency'][0.50] else 2 if x <= quantiles['Frequency'][0.75] else 1)
rfm['M_Score'] = rfm['MonetaryValue'].apply(rfm_score, args=('MonetaryValue', quantiles,))

# Combine RFM scores
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Display the final RFM segmentation
print(rfm.head())

               Recency  Frequency  MonetaryValue  R_Score  F_Score  M_Score  \
AccountNumber                                                                 
1000.0               0       1710       28698.51        1        1        3   
1002.0               0       1043       15622.97        1        3        1   
1006.0               0       1841       18778.19        1        1        2   
1018.0               1        805       15612.21        4        4        1   
1053.0               0       1319       19444.57        1        2        2   

              RFM_Score  
AccountNumber            
1000.0              113  
1002.0              131  
1006.0              112  
1018.0              441  
1053.0              122  


Combining the RFM table and the Original table

In [4]:
merged_df = pd.merge(df2, rfm, on='AccountNumber', how='left')

print(merged_df.head())

   AccountNumber  MoneyTransferAmount TransferredLocationAccount  \
0        10371.0                 4.00                     CINEMA   
1        98994.0                 2.65                COFFEE_SHOP   
2        98994.0                 2.20        A_LOCAL_COFFEE_SHOP   
3        34189.0                14.50                        BAR   
4        86417.0                 4.35                COFFEE_SHOP   

  TransactionDate CategoryTransferredLocationAccount  Recency  Frequency  \
0      2025-01-01                               FILM        0       1653   
1      2025-01-01                             COFFEE        1        960   
2      2025-01-01                             COFFEE        1        960   
3      2025-01-01                                BAR        1        874   
4      2025-01-01                             COFFEE        0       1692   

   MonetaryValue  R_Score  F_Score  M_Score RFM_Score  
0       23974.11        1        1        3       113  
1       13433.62      

In [6]:
#find AVG money spend for each category for each customers
avg_spend = merged_df.groupby(['AccountNumber', 'CategoryTransferredLocationAccount'])['MoneyTransferAmount'].mean().reset_index()

#Identifying the category with the highest average spend for each customer
AccountNumber_highestAvgSpend_df = avg_spend.loc[avg_spend.groupby('AccountNumber')['MoneyTransferAmount'].idxmax()]

# Check if each AccountNumber appears only once
# unique_accounts = AccountNumber_highestAvgSpend_df['AccountNumber'].nunique() == AccountNumber_highestAvgSpend_df.shape[0]
# print(f"Each AccountNumber appears only once: {unique_accounts}")
# 
#Create a new column by mapping values from the 'key' column using the mapping dictionary
mapped_dict_lovers = {entry['Value']: entry['Main_Category'] for entry in mapping_dict}
mapped_dict_lovers['Numeric'] = 'Numeric'
AccountNumber_highestAvgSpend_df['WillingnessToSpend'] = AccountNumber_highestAvgSpend_df['CategoryTransferredLocationAccount'].map(mapped_dict_lovers)
result = pd.merge(merged_df, AccountNumber_highestAvgSpend_df[['AccountNumber','WillingnessToSpend']], on='AccountNumber', how='left')

print(result.head())
# print(result[result['AccountNumber'] == 55938])


   AccountNumber  MoneyTransferAmount TransferredLocationAccount  \
0        10371.0                 4.00                     CINEMA   
1        98994.0                 2.65                COFFEE_SHOP   
2        98994.0                 2.20        A_LOCAL_COFFEE_SHOP   
3        34189.0                14.50                        BAR   
4        86417.0                 4.35                COFFEE_SHOP   

  TransactionDate CategoryTransferredLocationAccount  Recency  Frequency  \
0      2025-01-01                               FILM        0       1653   
1      2025-01-01                             COFFEE        1        960   
2      2025-01-01                             COFFEE        1        960   
3      2025-01-01                                BAR        1        874   
4      2025-01-01                             COFFEE        0       1692   

   MonetaryValue  R_Score  F_Score  M_Score RFM_Score WillingnessToSpend  
0       23974.11        1        1        3       113      